In [12]:
import numpy as np
import pandas as pd
import math

from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.auto import tqdm

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
    
print(f'Actual device: {device}')

Actual device: mps


In [3]:
X = load_iris()['data']
X
min_max = MinMaxScaler()
X = min_max.fit_transform(X)
X[:10]

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ]])

In [4]:
class IrisDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype = torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

dataset = IrisDataset(X)
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

for batch in dataloader:
    print(f'{batch.shape}\n{batch}')
    break

torch.Size([32, 4])
tensor([[0.6667, 0.5417, 0.7966, 1.0000],
        [0.1111, 0.5000, 0.0508, 0.0417],
        [0.6667, 0.4167, 0.7119, 0.9167],
        [0.2500, 0.2917, 0.4915, 0.5417],
        [0.1944, 0.5833, 0.1017, 0.1250],
        [0.5000, 0.4167, 0.6610, 0.7083],
        [0.1667, 0.4583, 0.0847, 0.0000],
        [0.3056, 0.7917, 0.0508, 0.1250],
        [0.9444, 0.3333, 0.9661, 0.7917],
        [0.3333, 0.1667, 0.4576, 0.3750],
        [0.3333, 0.2500, 0.5763, 0.4583],
        [0.2222, 0.7500, 0.1017, 0.0417],
        [0.1667, 0.2083, 0.5932, 0.6667],
        [0.8611, 0.3333, 0.8644, 0.7500],
        [0.3889, 0.3333, 0.5254, 0.5000],
        [0.1667, 0.4167, 0.0678, 0.0417],
        [0.6111, 0.5000, 0.6949, 0.7917],
        [0.7222, 0.4583, 0.6949, 0.9167],
        [0.1944, 0.4167, 0.1017, 0.0417],
        [0.0278, 0.5000, 0.0508, 0.0417],
        [0.5556, 0.5417, 0.6271, 0.6250],
        [0.7222, 0.4583, 0.7458, 0.8333],
        [0.3333, 0.1250, 0.5085, 0.5000],
        [0.305

In [93]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, 
                 enc_input_dim = 4, 
                 encoder_layers = 10,
                 encoder_max_width = 1024,
                 latent_space_dim = 4):
        super(VariationalAutoEncoder, self).__init__()
        
        encoder_layers_width = [neurons for neurons in range(max_width, latent_space_dim, -int(encoder_max_width/encoder_layers))]
        encoder_layers_width.insert(0, enc_input_dim)
        encoder_layers_width.append(latent_space_dim)
        
        encoder_layers = []
        
        for i in range(len(encoder_layers_width) - 1):
            
            input_dim = encoder_layers_width[i]
            output_dim = encoder_layers_width[i + 1]
            
            encoder_layers.append(nn.Linear(input_dim, output_dim))
            
            if i != len(encoder_layers_width) - 2:
                encoder_layers.append(nn.ReLU())
                
        self.Encoder = nn.Sequential(*encoder_layers)
        
vae = VariationalAutoEncoder()
vae

VariationalAutoEncoder(
  (Encoder): Sequential(
    (0): Linear(in_features=4, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=922, bias=True)
    (3): ReLU()
    (4): Linear(in_features=922, out_features=820, bias=True)
    (5): ReLU()
    (6): Linear(in_features=820, out_features=718, bias=True)
    (7): ReLU()
    (8): Linear(in_features=718, out_features=616, bias=True)
    (9): ReLU()
    (10): Linear(in_features=616, out_features=514, bias=True)
    (11): ReLU()
    (12): Linear(in_features=514, out_features=412, bias=True)
    (13): ReLU()
    (14): Linear(in_features=412, out_features=310, bias=True)
    (15): ReLU()
    (16): Linear(in_features=310, out_features=208, bias=True)
    (17): ReLU()
    (18): Linear(in_features=208, out_features=106, bias=True)
    (19): ReLU()
    (20): Linear(in_features=106, out_features=4, bias=True)
  )
)